# Predittore di precipitazione nella città di Udine (S. Osvaldo)

Questo programma aiuta a predirre la situazione di pioggia del giorno sucessivo partendo da alcuni dati del giorno attuale.

Nello specifico, utilizza:
* mm di Pioggia di oggi
* Temperature (massima, minima, media)
* Umidità (massima, media)
* Anno, Mese e Giorno (come posizione nel mese)
* Vento (massimo, medio, direzione)
* Irradiamento solare (in KJ/m2)
* Pressione atmosferica media (in Pascal)

L'output invece sarà una stringa, che sarà:
* "Nulla o Minima" (<= 0.6 mm / giorno)
* "Presente" (> 0.6 mm / giorno)

Il valore di sbarramento di 0.6 mm / giorno è stato selezionato al posto del più intuitivo 0.0 mm / giorno come compromesso per evitare il rumore casuale che può influenzare la misurazione.

Per chi è più esperto, è possibile anche modificare il codice provando altri modelli con altri parametri per ottenere risultati diversi

Per iniziare, importeremo alcuni moduli necessari per il funzionamento dei modelli. Sucessivamente importeremo il file contente i dati relativi al passato meteorologico, che per comodità è già stato pulito

In [116]:
import sklearn
import pandas as pd
import numpy as np
import joblib

df = pd.read_parquet("https://raw.githubusercontent.com/FreyFlyy/data-science-files/main/Udine_dati_meteo.parquet")

Ora andremo a definire le varie colonne del dataframe, distinguendole tra input e output, e tra numeriche e categoriche, automaticamente.
Andremo anche a creare una lista di colonne da ignorare durante la fase di training, che però in questo caso è vuota dato che useremo tutte le variabili

In [117]:
cols_to_ignore = []
cols_to_ignore.append("PioggiaDomani")

input_cols = df.columns.drop(cols_to_ignore)
output_cols = ["PioggiaDomani"]

numerical_cols = df[input_cols].select_dtypes(include=[np.number]).columns
categorical_cols = df[input_cols].select_dtypes(include=["object"]).columns

Dopo aver definito le colonne, procederemo a trasformare tutti i dati:
* Andremo a riempire i valori vuoti di ogni colonna con SimpleImputer di scikit-learnì
* Poi, trasformeremo le variabili categoriche (non numeriche) in un formato leggibile dai modelli tramite OneHotEncoding

In questo caso non standardizzeremo i dati, essendo che useremo un modello ad albero, che per come sono strutturati non hanno bisogno di modifiche ulteriori

In [118]:
imputer = sklearn.impute.SimpleImputer(strategy="mean")
imputer.fit(df[numerical_cols])
df[numerical_cols] = imputer.transform(df[numerical_cols])

encoder = sklearn.preprocessing.OneHotEncoder()
encoder.fit(df[categorical_cols])
encoded = encoder.transform(df[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_cols), index=df.index)
df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

Una volta preparati i dati, li possiamo dividere in due set:
* Uno di training (da usare per addestrare il modello, 80% del totale)
* E uno di test (per testare il modello e quanto affidabile è, 20% del totale)

In [119]:
input_cols = df.columns.drop(cols_to_ignore)
output_cols = ["PioggiaDomani"]

df_train = df[:int(len(df)*0.8)]
df_test = df[int(len(df)*0.8):]

train_input = df_train[input_cols]
train_output = df_train[output_cols]
test_input = df_test[input_cols]
test_output = df_test[output_cols]

Essendo i casi di training sbilanciati, procederemo a specificare i diversi pesi da dare ai due casi di pioggia ("Nulla o Minima" e "Presente") in linea con le loro proporzioni
(Il fattore 1.0327 è stato messo per bilanciare le metriche di affidabilità che andremo a vedere poi)

In [120]:
factor = 1.0327

nul_weight = (df.shape[0] / df[df["PioggiaDomani"] == "Nulla o Minima"].shape[0])**factor
pres_weight = (df.shape[0] / df[df["PioggiaDomani"] == "Presente"].shape[0])**factor

cl_wght = {"Nulla o Minima": nul_weight, "Presente": pres_weight}

proporzioni = df["PioggiaDomani"].value_counts()
print(f"# Proporzioni nei dati reali (non omogenee)\n\n- Casi di giornata limpida 🌥️: {proporzioni.iloc[0]}\n- Casi di pioggia 🌧️: {proporzioni.iloc[1]}")

# Proporzioni nei dati reali (non omogenee)

- Casi di giornata limpida 🌥️: 2469
- Casi di pioggia 🌧️: 1018


Dopo aver sperimentato diverse combinazioni tra la profondità dell'albero (max_depth) e il coefficiente di riequilibrio delle classi (factor), abbiamo identificato che i valori max_depth = 2, factor = 1.0327 rappresentano un buon compromesso, ma non per forza il migliore

Questo punto ottimizza la distanza tra i recall delle due classi, mantenendo il modello semplice e interpretabile



---



Ora passiamo alla creazione del modello.
Useremo come anticipato un modello ad albero decisionale di tipo ensembie, chiamaro RandomForestClassifier, da scikit-learn
A differenza di modelli non ensembie, questi creano diversi modelli individuali con caratteristiche variegate, per poi unirli insieme sfruttando lo smorzamento degli errori dei singoli modelli rapportati alla loro interità

In [121]:
model = sklearn.ensemble.RandomForestClassifier(max_depth = 2, random_state = 42, bootstrap = True, class_weight = {"Nulla o Minima": nul_weight, "Presente": pres_weight})
model.fit(train_input, train_output.values.ravel())

RandomForestClassifier(class_weight={'Nulla o Minima': 1.428346570864554,
                                     'Presente': 3.5660628712917144},
                       max_depth=2, random_state=42)

Questi risultati saranno quindi salvati ed usati per creare una matrice di confusione, che ci mostrerà dove i nostri modelli sono più accurati, e dove invece non lo sono

Semplificandola, valori più vicini ad 1 significheranno predizioni più accurate, mentre valori vicini a 0 significheranno predizioni quasi mai giuste

Creeremo inoltre una tabella che raccoglie, per ogni range di confidenza di una predizione (i.e. quanto il modello è "sicuro" della scelta), la sua affidabilità media: ci servirà in futuro


In [122]:
def prob_class(prob):
    if 0.5 <= prob < 0.6:
        return "0.5 - 0.6"
    elif 0.6 <= prob < 0.7:
        return "0.6 - 0.7"
    elif 0.7 <= prob < 0.8:
        return "0.7 - 0.8"
    elif 0.8 <= prob < 0.9:
        return "0.8 - 0.9"
    elif 0.9 <= prob <= 1:
        return "0.9 - 1"
    else:
        return "<0.5"

test_output_pred = model.predict(test_input)
test_output_proba = model.predict_proba(test_input)
massimi = np.max(test_output_proba, axis=1)


output_df = pd.DataFrame({"Guess": test_output_pred, "Probability": massimi, "TrueLabel": test_output.values.ravel()})

output_df["Right"] = output_df["Guess"] == output_df["TrueLabel"]

output_df["Conf_class"] = output_df["Probability"].apply(prob_class)

recall_per_class = output_df.groupby("Conf_class")["Right"].mean().reset_index(name="Recall")

conf_matx = sklearn.metrics.confusion_matrix(test_output, test_output_pred, normalize="true", labels=["Nulla o Minima", "Presente"])

nulla_accuracy = conf_matx[0][0]
presente_accuracy = conf_matx[1][1]

print(f"### PERCENTUALI DI ACCURATEZZA STORICA PER OGNI MODELLO:\n")
print(f"\n# Modello trainato (recalls):\n\n- Nulla o Minima 🌥️ {round(nulla_accuracy*100, 2)}%\n- Presente 🌧️ {round(presente_accuracy*100, 2)}%\n\n")
accuracy = sklearn.metrics.accuracy_score(test_output, test_output_pred)
print(f"Accuratezza totale sui dati: {round(accuracy*100, 2)}%")
y_true_bin = (test_output == "Presente").astype(int).values.ravel()
y_scores = model.predict_proba(test_input)[:, 1]
roc_auc = sklearn.metrics.roc_auc_score(y_true_bin, y_scores)
print(f"ROC-AUC score: {round(roc_auc*100, 2)}%")
precision = sklearn.metrics.precision_score(test_output, test_output_pred, pos_label="Presente")
print(f"Precisione totale sui dati: {round(precision*100, 2)}%")
f1 = sklearn.metrics.f1_score(test_output, test_output_pred, pos_label="Presente")
print(f"F1 score sui dati: {round(f1*100, 2)}%")

### PERCENTUALI DI ACCURATEZZA STORICA PER OGNI MODELLO:


# Modello trainato (recalls):

- Nulla o Minima 🌥️ 71.28%
- Presente 🌧️ 71.5%


Accuratezza totale sui dati: 71.35%
ROC-AUC score: 77.27%
Precisione totale sui dati: 52.4%
F1 score sui dati: 60.47%


Senza andare troppo nel dettaglio, da queste metriche capiamo che tra tutte le giornate di un certo tipo, il nostro modello riesce a prevedere correttamente circa il 71.35% dei casi

Andiamo quindi a creare una nuova funzione, con lo scopo di predirre singole giornate a partire dai dati del giorno precedente

In [123]:
def predict_single_day(input):
  df_input = pd.DataFrame(input, columns = input.keys(), index=[1])

  numerical_cols = df_input.select_dtypes(include=[np.number]).columns.tolist()
  categorical_cols = df_input.select_dtypes(["object"]).columns.tolist()


  df_input[numerical_cols] = imputer.transform(df_input[numerical_cols])

  encoded = encoder.transform(df_input[categorical_cols]).toarray()
  encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_cols), index=df_input.index)
  df_input = pd.concat([df_input.drop(columns=categorical_cols), encoded_df], axis=1)


  pred = model.predict(df_input)[0]
  conf = model.predict_proba(df_input)[0]
  max_prob = max(conf)

  pred_descriptive = "Presente 🌧️ (> 2mm totali) " if pred == "Presente" else "Nulla o Minima 🌥️ (< 2mm totali)"
  acc = presente_accuracy if pred == "Presente" else nulla_accuracy

  conf_class = prob_class(max_prob)
  acc_class = recall_per_class.loc[recall_per_class["Conf_class"] == conf_class, "Recall"].values[0]

  print(f"Domani la pioggia sarà {pred_descriptive.lower()}")
  print(f"Confidenza oggi: {round(100*acc_class, 1)}%")

Ora possiamo immettere i dati della giornata in questione nel codice sotto, e vedere la previsione per la giornata sucessiva

In [125]:
input_to_predict = {
    "Anno": 2025, # Anno della predizione
    "Mese": "Ago", # Mese della predizione (abbreviato, 3 lettere, prima lettera maiuscola)
    "PosizioneMese": "Meta", # In che fase del mese si trova ("Inizio", "Meta", "Fine")
    "DirVentoMax": "S", # Direzione cardinale del vento massima velocità (Anche combinazioni es. "SO")
    "TempMin [°C]": 18.2, # Temperatura minima registrata (°C)
    "TempMed [°C]": 29.1, # Temperatura media registrata (°C)
    "TempMax [°C]": 33.6, # Temperatura massima registrata (°C)
    "UmiditaMed [%]": 80, # Umidità media registrata (%)
    "UmiditaMax [%]": 91, # Umidità massima registrata (%)
    "VentoMed [km/h]": 6, # Velocità media del vento (km/h)
    "VentoMax [km/h]": 23, # Velocità massima del vento (km/h)
    "Radiazione [KJ/m2]": 11270, # Irradiamento solare totale [KJ/m2]
    "Pressione [Pa]": 101635, # Pressiona atmosferica media (Pascal)
    "mmPioggia": 0 # Millimetri di pioggia totali (anche scritti come Litri per m2)
}

predict_single_day(input_to_predict)

Domani la pioggia sarà nulla o minima 🌥️ (< 2mm totali)
Confidenza oggi: 66.7%


Se si vuole, si possono anche modificare i dati a proprio piacimento, modificando i valori tra i due punti (:) e la virgola (,) seguendo le unità di misura quando questi sono di tipo numerico.

Quando invece sono di tipo testuale (stringa), ricordarsi di racchiudere sempre il testo tra le virgolette (" ")

OPZIONALE: reso il modello (.pkl) e il dataset (.csv) accessibile e scaricabile

In [126]:
modello_totale = {
    "modello_predittivo": model,
    "modello_encoder": encoder,
    "modello_imputer": imputer
}

joblib.dump(modello_totale, "modello_totale.pkl")
dataset = df.to_csv("dataset.csv")

# Conclusione

Questo modello non è completo, e non sarà nemmeno il più accurato o efficiente, tuttavia offre un buon punto di partenza per modellazioni meteorologiche su dataset pubblici.

L'accuratezza media è di solo 1% maggiore rispetto ad un banale "sempre no", tuttavia non è per questo che è stato ideato: ho privilegiato l'equilibrio tra giornate pioggia e giornate non.
Se avessi voluto solamente un'accuratezza massima, avrei sacrificato molto la categoria della pioggia, essendo meno frequente.

Spero possa servire utile, e spero abbia servito come prova del fatto che tramite programmi del genere è possibile fare cose molto più complesse di quelle che ci si può aspettare

-

*Scolz F.*